In [1]:
import pandas as pd
import plotly.express as px

In [2]:
df = pd.read_csv('data/crypto.csv')

Q2. Whats the most money you could have made in one day on a single Monero coin?  
  a) In absolute terms (e.g. you bought for £3000 one day and sold for £3300 the next day for £300 profit)  
  b) In percentage terms (e.g. you bought for £3000 one day and sold for £3300 the next day for a 10% profit)  

In [137]:
df_m = df[df['Asset_Name'] == 'Monero'].copy()

In [138]:
df_m['daily_diff'] = df_m['Close'].diff()
df_m['daily_pct'] = df_m['Close'].pct_change()

In [143]:
df_m.head()

,Asset_Name,timestamp,Close,daily_diff,daily_pct
9520,Monero,2018-01-01,329.460,NaN,NaN
9521,Monero,2018-01-02,340.345,10.885,0.033039
9522,Monero,2018-01-03,365.160,24.815,0.072911
9523,Monero,2018-01-04,385.030,19.870,0.054415
9524,Monero,2018-01-05,371.000,-14.030,-0.036439


In [141]:
df_m['daily_diff'].max()

81.36449999999996

In [144]:
df_m['daily_pct'].max()

0.39538019709604044

[Hard - *A version of this was actually one of my interview questions here* ~ Joe]  
Q4. If you could have bought one Litecoin coin on any date, and sold on any later date  
  a) What is the most money you could have made? What days would you choose?  
  b) What is the most money you could lose? What days would you choose?  
  c) Can you solve this problem using just one for loop?  
  d) If you had to buy and sell one of EVERY coin on the same date, which days would you chose?  

In [165]:
# Make things easier - take a data frame that just has Litecoin
df_lc = df[df['Asset_Name'] == 'Litecoin'].copy()

In [129]:
df_lc.head()

,Asset_Name,timestamp,Close
8160,Litecoin,2018-01-01,225.2067
8161,Litecoin,2018-01-02,223.9000
8162,Litecoin,2018-01-03,251.2860
8163,Litecoin,2018-01-04,243.2560
8164,Litecoin,2018-01-05,238.1320


In [166]:
df_lc = df_lc.reset_index()
df_lc = df_lc.drop('index', axis=1)
# df_lc.at[0, 'Close'] = 1000000

In [170]:
df_lc.shape

(1360, 3)

In [167]:
df_lc.head()

,Asset_Name,timestamp,Close
0,Litecoin,2018-01-01,225.2067
1,Litecoin,2018-01-02,223.9000
2,Litecoin,2018-01-03,251.2860
3,Litecoin,2018-01-04,243.2560
4,Litecoin,2018-01-05,238.1320


a) What is the most money you could have made? What days would you choose?

In [168]:
%%timeit -n 1 -r 1
max = -1E99, None
min = 1E99, None

max_profit = -99999

for ix, day in df_lc.iterrows():
    for ix2, day2 in df_lc[ix:].iterrows():
        profit = day2['Close'] - day['Close']
        if profit > max_profit:
            max_profit = profit
            max = day2['Close'], day2['timestamp']
            min = day['Close'], day['timestamp']

32.1 s ± 0 ns per loop (mean ± std. dev. of 1 run, 1 loop each)


In [126]:
print(max)
print(min)
print(max_profit)

(-1e+99, None)
(1e+99, None)
-99999


In [114]:
max = 1E99, None
min = -1E99, None

max_loss = 0

for ix, day in df_lc.iterrows():
    for ix2, day2 in df_lc[ix:].iterrows():
        loss = day2['Close'] - day['Close']
        if loss < max_loss:
            max_loss = loss
            max = day2['Close'], day2['timestamp']
            min = day['Close'], day['timestamp']

In [84]:
print(max)
print(min)
print(max_loss)

(107.2503, '2021-07-21')
(389.755, '2021-05-10')
-282.5047


In [123]:
%%timeit

max = -1E99, None
min = 1E99, None

max_profit = -99999

for ix, day in df_lc.iterrows():
    profit = day['Close'] - min[0]
    if profit > max_profit:
        max_profit = profit
        max = day['Close'], day['timestamp']
    if day['Close'] < min[0]:
        min = day['Close'], day['timestamp']

51.3 ms ± 1.37 ms per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [124]:
print(max)
print(min)
print(max_profit)

(-1e+99, None)
(1e+99, None)
-99999


In [104]:
df_piv = pd.pivot_table(df, index='timestamp', columns='Asset_Name', values='Close').fillna(0)

In [105]:
df_piv.head()

Asset_Name,Binance Coin,Bitcoin,Bitcoin Cash,Cardano,Dogecoin,EOS.IO,Ethereum,Ethereum Classic,IOTA,Litecoin,Maker,Monero,Stellar,TRON
timestamp,,,,,,,,,,,,,,
2018-01-01,8.5300,13850.176,2374.5900,0.0,0.0,7.6576,738.5075,25.877,0.0,225.2067,0.0,329.460,0.0,0.0
2018-01-02,8.4381,13468.698,2323.3200,0.0,0.0,7.6401,757.3900,29.531,0.0,223.9000,0.0,340.345,0.0,0.0
2018-01-03,8.6996,14743.424,2551.2850,0.0,0.0,8.3090,859.6750,31.871,0.0,251.2860,0.0,365.160,0.0,0.0
2018-01-04,9.2880,15107.792,2557.6467,0.0,0.0,9.4053,935.4225,33.016,0.0,243.2560,0.0,385.030,0.0,0.0
2018-01-05,9.1215,15158.492,2354.4433,0.0,0.0,9.4039,941.2525,33.997,0.0,238.1320,0.0,371.000,0.0,0.0


In [109]:
df_piv['Total_Val'] = df_piv[df_piv.columns].sum(axis=1)

In [110]:
df_piv

Asset_Name,Binance Coin,Bitcoin,Bitcoin Cash,Cardano,Dogecoin,EOS.IO,Ethereum,Ethereum Classic,IOTA,Litecoin,Maker,Monero,Stellar,TRON,Total_Val
timestamp,,,,,,,,,,,,,,,
2018-01-01,8.5300,13850.1760,2374.5900,0.0000,0.0000,7.6576,738.5075,25.8770,0.0000,225.2067,0.0000,329.4600,0.0000,0.0000,17560.0048
2018-01-02,8.4381,13468.6980,2323.3200,0.0000,0.0000,7.6401,757.3900,29.5310,0.0000,223.9000,0.0000,340.3450,0.0000,0.0000,17159.2622
2018-01-03,8.6996,14743.4240,2551.2850,0.0000,0.0000,8.3090,859.6750,31.8710,0.0000,251.2860,0.0000,365.1600,0.0000,0.0000,18819.7096
2018-01-04,9.2880,15107.7920,2557.6467,0.0000,0.0000,9.4053,935.4225,33.0160,0.0000,243.2560,0.0000,385.0300,0.0000,0.0000,19280.8565
2018-01-05,9.1215,15158.4920,2354.4433,0.0000,0.0000,9.4039,941.2525,33.9970,0.0000,238.1320,0.0000,371.0000,0.0000,0.0000,19115.8422
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2021-09-17,424.4520,47769.5096,638.9917,2.4177,0.2416,5.0204,3570.6356,57.9790,1.5004,185.5890,2967.6837,264.4540,0.3308,0.1161,55888.9216
2021-09-18,406.8785,47333.4914,621.2033,2.3515,0.2401,5.2466,3401.4414,56.7022,1.7114,180.0112,2791.8906,262.5400,0.3201,0.1055,55064.1338
2021-09-19,410.5245,48295.5087,630.5300,2.3700,0.2413,5.4658,3433.6931,57.1550,1.6402,181.3519,2840.2917,271.6333,0.3220,0.1059,56130.8334


In [113]:
max = -1E99, None
min = 1E99, None

max_profit = -99999

for ix, day in df_piv.iterrows():
    profit = day['Total_Val'] - min[0]
    if profit > max_profit:
        max_profit = round(profit, 2)
        max = round(day['Total_Val'], 2), ix
    if day['Total_Val'] < min[0]:
        min = round(day['Total_Val'], 2), ix

print(max)
print(min)
print(max_profit)

(71683.58, '2021-04-16')
(3798.37, '2018-12-16')
67885.21
